아래 링크에서 각 셀의 output을 확인하실 수 있습니다. <br>
https://colab.research.google.com/drive/1dsAbklolKaRJkz4Qwf0KOfE8SbfEFX3c?usp=sharing

In [ ]:
import zipfile
import os

zip_path = "/content/SeatMap.v1i.yolov7pytorch.zip"
extract_dir = "/content/SeatMap.v1i.yolov7pytorch"

# 압축 해제
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 기존 데이터 경로
base_path = '/content/SeatMap.v1i.yolov7pytorch/SeatMap.v1i.yolov7pytorch'
img_dir = os.path.join(base_path, 'train/images')
lbl_dir = os.path.join(base_path, 'train/labels')

# 출력 경로
base_path = '/content/SeatMap'
train_img_dir = os.path.join(base_path, 'images/train')
val_img_dir   = os.path.join(base_path, 'images/val')
train_lbl_dir = os.path.join(base_path, 'labels/train')
val_lbl_dir   = os.path.join(base_path, 'labels/val')

# 디렉토리 생성
for d in [train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
    os.makedirs(d, exist_ok=True)

# 이미지 파일 리스트
img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))]

# 학습/검증 분할
train_imgs, val_imgs = train_test_split(img_files, test_size=0.2, random_state=42)

def move_data(file_list, src_img_dir, src_lbl_dir, dst_img_dir, dst_lbl_dir):
    for f in file_list:
        img_src = os.path.join(src_img_dir, f)
        lbl_src = os.path.join(src_lbl_dir, os.path.splitext(f)[0] + '.txt')

        img_dst = os.path.join(dst_img_dir, f)
        lbl_dst = os.path.join(dst_lbl_dir, os.path.splitext(f)[0] + '.txt')

        shutil.copyfile(img_src, img_dst)
        if os.path.exists(lbl_src):
            shutil.copyfile(lbl_src, lbl_dst)

# 이동 실행
move_data(train_imgs, img_dir, lbl_dir, train_img_dir, train_lbl_dir)
move_data(val_imgs, img_dir, lbl_dir, val_img_dir, val_lbl_dir)

print("기존 이미지/라벨 → train/val 분할 완료")

In [ ]:
import os

# 디렉토리 경로 설정
base_dir = '/content/SeatMap/images'
subdirs = ['train', 'val']

# 이미지 확장자 정의 (YOLO에서 주로 사용되는 포맷들)
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# 각 폴더에 있는 이미지 개수 출력
for subdir in subdirs:
    folder_path = os.path.join(base_dir, subdir)
    image_count = len([
        f for f in os.listdir(folder_path)
        if os.path.splitext(f)[1].lower() in image_extensions
    ])
    print(f"{subdir} 이미지 개수: {image_count}장")

In [ ]:
import os
import cv2
import random
import albumentations as A
from tqdm import tqdm

# 디렉토리 설정
input_img_dir = '/content/SeatMap/images/train'
input_lbl_dir = '/content/SeatMap/labels/train'

output_img_dir = '/content/augmented/images'
output_lbl_dir = '/content/augmented/labels'
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_lbl_dir, exist_ok=True)

# 클래스 수 (YOLO format assumes class indices)
class_names = ['empty', 'occupied']  # optional, for verification only

# 증강 파이프라인 (bbox 처리 포함)
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.GaussNoise(p=0.3),
    A.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), p=0.5),
],
bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.3))

# 증강 횟수
AUG_PER_IMAGE = 2

image_files = [f for f in os.listdir(input_img_dir) if f.endswith(('.jpg', '.png'))]

for img_file in tqdm(image_files):
    img_path = os.path.join(input_img_dir, img_file)
    label_path = os.path.join(input_lbl_dir, os.path.splitext(img_file)[0] + '.txt')

    # 이미지 로드
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w = image.shape[:2]

    # 라벨 로드
    bboxes = []
    class_labels = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                cls, x, y, bw, bh = map(float, line.strip().split())
                bboxes.append([x, y, bw, bh])
                class_labels.append(int(cls))
    else:
        continue  # 라벨 없으면 스킵

    # 증강 반복
    for i in range(AUG_PER_IMAGE):
        augmented = augment(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_img = augmented['image']
        aug_bboxes = augmented['bboxes']
        aug_labels = augmented['class_labels']

        # 파일명 설정
        base_name = os.path.splitext(img_file)[0]
        aug_img_name = f"{base_name}_aug{i}.jpg"
        aug_lbl_name = f"{base_name}_aug{i}.txt"

        # 이미지 저장
        aug_img_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(output_img_dir, aug_img_name), aug_img_bgr)

        # 라벨 저장
        with open(os.path.join(output_lbl_dir, aug_lbl_name), 'w') as f:
            for label, box in zip(aug_labels, aug_bboxes):
                f.write(f"{label} {box[0]:.6f} {box[1]:.6f} {box[2]:.6f} {box[3]:.6f}\n")

In [ ]:
import os
import random
import cv2
from google.colab.patches import cv2_imshow

# 증강된 이미지 및 라벨 경로
aug_img_dir = '/content/augmented/images'
aug_lbl_dir = '/content/augmented/labels'

# 클래스 이름 정의 (YOLO 라벨 인덱스 기준)
class_names = ['empty', 'occupied']  # 반드시 YOLO 클래스 순서와 일치

# 시각화할 이미지 수
NUM_SAMPLES = 5

# 이미지 파일 리스트
image_files = [f for f in os.listdir(aug_img_dir) if f.endswith(('.jpg', '.png'))]

# 무작위 샘플 선택
for img_file in random.sample(image_files, min(NUM_SAMPLES, len(image_files))):
    img_path = os.path.join(aug_img_dir, img_file)
    label_path = os.path.join(aug_lbl_dir, os.path.splitext(img_file)[0] + '.txt')

    image = cv2.imread(img_path)
    h, w = image.shape[:2]

    if not os.path.exists(label_path):
        print(f"라벨 없음: {label_path}")
        continue

    # 라벨 읽기 및 바운딩박스 시각화
    with open(label_path, 'r') as f:
        for line in f:
            cls, x, y, bw, bh = map(float, line.strip().split())
            x1 = int((x - bw / 2) * w)
            y1 = int((y - bh / 2) * h)
            x2 = int((x + bw / 2) * w)
            y2 = int((y + bh / 2) * h)
            color = (0, 255, 0) if int(cls) == 0 else (0, 0, 255)
            label = class_names[int(cls)]
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    print(f"시각화: {img_file}")
    cv2_imshow(image)

In [ ]:
import os
import shutil

# 소스 디렉토리 목록
image_sources = [
    '/content/SeatMap/images/train',
    '/content/augmented/images'
]

label_sources = [
    '/content/SeatMap/labels/train',
    '/content/augmented/labels'
]

# 대상 디렉토리
merged_image_dir = '/content/merged/images'
merged_label_dir = '/content/merged/labels'

# 확장자 설정
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
label_extension = '.txt'

# 대상 디렉토리 생성
os.makedirs(merged_image_dir, exist_ok=True)
os.makedirs(merged_label_dir, exist_ok=True)

# 이미지 + 라벨 병합
for img_src, lbl_src in zip(image_sources, label_sources):
    for filename in os.listdir(img_src):
        if os.path.splitext(filename)[1].lower() in image_extensions:
            base_name, ext = os.path.splitext(filename)

            src_img_path = os.path.join(img_src, filename)
            src_lbl_path = os.path.join(lbl_src, f"{base_name}{label_extension}")

            dst_img_path = os.path.join(merged_image_dir, filename)
            dst_lbl_path = os.path.join(merged_label_dir, f"{base_name}{label_extension}")

            # 이름 중복 방지
            counter = 1
            while os.path.exists(dst_img_path) or os.path.exists(dst_lbl_path):
                new_base = f"{base_name}_{counter}"
                dst_img_path = os.path.join(merged_image_dir, f"{new_base}{ext}")
                dst_lbl_path = os.path.join(merged_label_dir, f"{new_base}{label_extension}")
                counter += 1

            shutil.copy2(src_img_path, dst_img_path)

            if os.path.exists(src_lbl_path):
                shutil.copy2(src_lbl_path, dst_lbl_path)

print("모든 이미지와 라벨이 /content/merged 디렉토리로 병합 완료됨")

In [ ]:
import os

# 경로 설정
image_dir = '/content/merged/images'
label_dir = '/content/merged/labels'

# 확장자 설정
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
label_extension = '.txt'

# 이미지 개수 세기
image_count = len([
    f for f in os.listdir(image_dir)
    if os.path.splitext(f)[1].lower() in image_extensions
])

# 라벨 개수 세기
label_count = len([
    f for f in os.listdir(label_dir)
    if f.endswith(label_extension)
])

print(f"이미지 수: {image_count}장")
print(f"라벨 수: {label_count}개")

In [ ]:
!pip install roboflow -q

from roboflow import Roboflow
rf = Roboflow(api_key="hrOWKFYjtwFdmqux4eNI")
project = rf.workspace("protective-equipment-detection-in-manufacturing-industry").project("chair-occupied-unoccupied")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
import os
from glob import glob

label_dirs = [
    "chair-occupied-/-unoccupied-1/train/labels",
    "chair-occupied-/-unoccupied-1/valid/labels",
    "chair-occupied-/-unoccupied-1/test/labels"
]

def step1_handle_single_0_convert_to_1(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()

    class_ids = {int(line.strip().split()[0]) for line in lines}
    if class_ids == {0}:  # 오직 0만 있는 경우
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            parts[0] = '1'
            new_lines.append(' '.join(parts) + '\n')
        with open(label_path, 'w') as f:
            f.writelines(new_lines)
        return True
    return False

def step2_convert_0_to_2_if_mixed(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()

    class_ids = {int(line.strip().split()[0]) for line in lines}
    if 0 in class_ids and len(class_ids) > 1:  # 0과 다른 라벨이 섞여있을 때
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts[0] == '0':
                parts[0] = '2'
            new_lines.append(' '.join(parts) + '\n')
        with open(label_path, 'w') as f:
            f.writelines(new_lines)
        return True
    return False

def step3_convert_2_to_0(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()
    changed = False
    new_lines = []
    for line in lines:
        parts = line.strip().split()
        if parts[0] == '2':
            parts[0] = '0'
            changed = True
        new_lines.append(' '.join(parts) + '\n')
    if changed:
        with open(label_path, 'w') as f:
            f.writelines(new_lines)
    return changed

def step4_swap_0_and_1(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()
    changed = False
    new_lines = []
    for line in lines:
        parts = line.strip().split()
        if parts[0] == '0':
            parts[0] = '1'
            changed = True
        elif parts[0] == '1':
            parts[0] = '0'
            changed = True
        new_lines.append(' '.join(parts) + '\n')
    if changed:
        with open(label_path, 'w') as f:
            f.writelines(new_lines)
    return changed

# 실행
for label_dir in label_dirs:
    label_paths = glob(os.path.join(label_dir, "*.txt"))

    for path in label_paths:
        step1_handle_single_0_convert_to_1(path)
        step2_convert_0_to_2_if_mixed(path)
        step3_convert_2_to_0(path)
        step4_swap_0_and_1(path)

print("🔄 모든 전처리 단계 완료.")

In [ ]:
import os
from glob import glob
import matplotlib.pyplot as plt
import cv2

# 기준 디렉토리 (필요 시 valid/test 추가 가능)
base_dir = "chair-occupied-/-unoccupied-1/train"
label_dir = os.path.join(base_dir, "labels")
image_dir = os.path.join(base_dir, "images")

# 라벨에 따라 분류
only_0_files = []
mixed_01_files = []

label_paths = glob(os.path.join(label_dir, "*.txt"))

for label_path in label_paths:
    with open(label_path, "r") as f:
        lines = f.readlines()
    class_ids = {int(line.strip().split()[0]) for line in lines}

    filename = os.path.splitext(os.path.basename(label_path))[0]
    img_path = os.path.join(image_dir, filename + ".jpg")

    if not os.path.exists(img_path):
        continue

    if class_ids == {0}:
        only_0_files.append((img_path, label_path))
    elif class_ids == {0, 1} or class_ids == {1, 0}:
        mixed_01_files.append((img_path, label_path))

    if len(only_0_files) >= 10 and len(mixed_01_files) >= 10:
        break

# 박스 그리기 함수
def draw_boxes(img_path, label_path):
    image = cv2.imread(img_path)
    h, w = image.shape[:2]
    with open(label_path, "r") as f:
        for line in f:
            cls, x, y, bw, bh = map(float, line.strip().split())
            x1 = int((x - bw / 2) * w)
            y1 = int((y - bh / 2) * h)
            x2 = int((x + bw / 2) * w)
            y2 = int((y + bh / 2) * h)
            color = (0, 255, 0) if int(cls) == 0 else (0, 0, 255)
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, f"class {int(cls)}", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

# 시각화 함수
def show_samples(sample_list, title):
    plt.figure(figsize=(20, 10))
    plt.suptitle(title, fontsize=20)
    for idx, (img_path, label_path) in enumerate(sample_list[:10]):
        img = draw_boxes(img_path, label_path)
        plt.subplot(2, 5, idx + 1)
        plt.imshow(img)
        plt.title(os.path.basename(img_path), fontsize=8)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# 시각화 실행
show_samples(only_0_files, "🟥 Class 0 only (empty)")
show_samples(mixed_01_files, "🟩 Class 0 and 1 mixed")

/content/final_dataset/
<br>├── train/
<br>│   ├── images/  ← 300장 무작위 + merged 전체
<br>│   └── labels/  ← 위 300장에 해당 라벨 + merged 전체
<br>└── val/
<br>    ├── images/  ← Beyond.v1i.yolov8/val 이미지 전체
<br>    └── labels/  ← Beyond.v1i.yolov8/val 라벨 전체

In [ ]:
import os
import shutil
import random

# 경로 설정
final_root = '/content/final_dataset'
train_images_dir = os.path.join(final_root, 'train/images')
train_labels_dir = os.path.join(final_root, 'train/labels')
val_images_dir   = os.path.join(final_root, 'val/images')
val_labels_dir   = os.path.join(final_root, 'val/labels')

# 원본 경로
src_aug_images = '/content/chair-occupied-/-unoccupied-1/train/images'
src_aug_labels = '/content/chair-occupied-/-unoccupied-1/train/labels'

src_merged_images = '/content/merged/images'
src_merged_labels = '/content/merged/labels'

src_val_images = '/content/SeatMap/images/val'
src_val_labels = '/content/SeatMap/labels/val'

# 디렉토리 생성
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# 이미지 확장자
image_exts = ['.jpg', '.jpeg', '.png', '.bmp']
label_ext = '.txt'

### 1. 무작위 이미지 300장 선택 + 라벨 복사
all_aug_images = [
    f for f in os.listdir(src_aug_images)
    if os.path.splitext(f)[1].lower() in image_exts
]

random_selected = random.sample(all_aug_images, 300)

for img_file in random_selected:
    base_name = os.path.splitext(img_file)[0]
    img_path = os.path.join(src_aug_images, img_file)
    lbl_path = os.path.join(src_aug_labels, base_name + label_ext)

    shutil.copy2(img_path, os.path.join(train_images_dir, img_file))
    if os.path.exists(lbl_path):
        shutil.copy2(lbl_path, os.path.join(train_labels_dir, base_name + label_ext))

### 2. merged의 모든 이미지 및 라벨 복사
for img_file in os.listdir(src_merged_images):
    if os.path.splitext(img_file)[1].lower() in image_exts:
        base_name = os.path.splitext(img_file)[0]
        shutil.copy2(os.path.join(src_merged_images, img_file),
                     os.path.join(train_images_dir, img_file))
        lbl_file = base_name + label_ext
        lbl_src = os.path.join(src_merged_labels, lbl_file)
        if os.path.exists(lbl_src):
            shutil.copy2(lbl_src, os.path.join(train_labels_dir, lbl_file))

### 3. val 데이터 복사 (이미지 & 라벨)
for img_file in os.listdir(src_val_images):
    if os.path.splitext(img_file)[1].lower() in image_exts:
        shutil.copy2(os.path.join(src_val_images, img_file),
                     os.path.join(val_images_dir, img_file))

for lbl_file in os.listdir(src_val_labels):
    if lbl_file.endswith(label_ext):
        shutil.copy2(os.path.join(src_val_labels, lbl_file),
                     os.path.join(val_labels_dir, lbl_file))

print("모든 데이터가 /content/final_dataset 에 정리 완료되었습니다.")

In [ ]:
import os

# 경로 설정
image_dir = '/content/final_dataset/train/images'
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# 이미지 개수 계산
image_count = len([
    f for f in os.listdir(image_dir)
    if os.path.splitext(f)[1].lower() in image_extensions
])

print(f"/content/final_dataset/train/images 안에 있는 이미지 수: {image_count}장")

In [ ]:
import yaml

# 클래스 목록: 실제 프로젝트에 맞게 수정
class_names = ['empty', 'occupied']  # 예시 클래스

yaml_data = {
    'train': '/content/final_dataset/train/images',
    'val': '/content/final_dataset/val/images',
    'names': {i: name for i, name in enumerate(class_names)},
    'nc': len(class_names)
}

# 저장 경로
yaml_path = '/content/final_dataset/dataset.yaml'

# YAML 파일 저장
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_data, f, default_flow_style=False)

print(f"dataset.yaml 생성 완료: {yaml_path}")

In [ ]:
!pip install -q ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # 또는 yolov8s.pt, yolov8m.pt 등
model.train(data="/content/final_dataset/dataset.yaml", epochs=100, imgsz=640, device=0)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 경로: YOLO 학습 결과 디렉토리
results_csv_path = "/content/runs/detect/train/results.csv"

# CSV 불러오기
assert os.path.exists(results_csv_path), f"CSV 파일 없음: {results_csv_path}"
df = pd.read_csv(results_csv_path)

# 기본 정보
epochs = df.index + 1  # 0부터 시작하므로 +1

# ✅ 그래프 그리기
plt.figure(figsize=(14, 6))

# ---- Accuracy / Precision / Recall ----
plt.subplot(1, 2, 1)
plt.plot(epochs, df["metrics/precision(B)"], label="Precision", marker='o')
plt.plot(epochs, df["metrics/recall(B)"], label="Recall", marker='o')
plt.plot(epochs, df["metrics/mAP50(B)"], label="mAP50", marker='o')
plt.title("Precision / Recall / mAP")
plt.xlabel("Epoch")
plt.ylabel("Metric Value")
plt.grid(True)
plt.legend()

# ---- Loss ----
plt.subplot(1, 2, 2)
plt.plot(epochs, df["train/box_loss"], label="Box Loss", marker='x')
plt.plot(epochs, df["train/cls_loss"], label="Class Loss", marker='x')
plt.plot(epochs, df["val/box_loss"], label="Val Box Loss", linestyle="--")
plt.plot(epochs, df["val/cls_loss"], label="Val Class Loss", linestyle="--")
plt.title("Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from ultralytics import YOLO

# 학습된 모델 로드 (best.pt 또는 last.pt 선택 가능)
model = YOLO("runs/detect/train/weights/best.pt")

# 검증 수행 (data.yaml 기반으로 val 세트 사용)
metrics = model.val()

# 결과 요약
print("검증 완료")
print(metrics)  # 내부적으로 mAP50, mAP50-95 등 포함

In [ ]:
import os
import random
import cv2
from google.colab.patches import cv2_imshow

# YOLO val 결과 이미지 경로
val_result_dir = "/content/runs/detect/val"

# 파일 리스트 불러오기
img_files = [f for f in os.listdir(val_result_dir) if f.endswith(('.jpg', '.png'))]

# 무작위 5장 선택
sample_imgs = random.sample(img_files, min(5, len(img_files)))

# 시각화
for f in sample_imgs:
    img_path = os.path.join(val_result_dir, f)
    img = cv2.imread(img_path)
    print(f"{f}")
    cv2_imshow(img)

In [ ]:
# 학습된 모델 로드 (best.pt 또는 last.pt 선택 가능)
model = YOLO("runs/detect/train/weights/best.pt")

# 모델 저장
save_path = "/content/best_seatmap_model.pt"
model.save(save_path)

print(f"모델이 저장되었습니다: {save_path}")